1. find an initialization of V s.t  SV = 0 and minimize L1 norm of V
2. improve sparsity of SV rows by combining the loss functions with given lambda, using ADMM

In [2]:
using JuMP
using GLPK
using ECOS
using MAT
using SparseArrays
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using DelimitedFiles
using COSMO
using CSV
using DataFrames
using DelimitedFiles

In [6]:
vars = matread("R4T1.mat")
S = vars["S"]
U = vars["U"]
L = vars["L"]
lambda = vars["lambda"]
n,d = size(S)
print(size(S))
print(size(L))

n,d = size(S) # n=10 d=100
d,k = size(L) # k = 5

print("done")

(2172, 4456)(4456, 50)done

In [9]:
X = zeros(size(L)[1],size(L)[2]);

n,d = size(S)

for j in 1:size(X)[2]
    model = Model(ECOS.Optimizer)
    @variable(model, L[i,j] <= x[i = 1:d] <= U[i,j])
    @constraint(model,S*x .== 0)
    @variable(model, x_abs[1:d])
    @constraint(model, x_abs .>= x)
    @constraint(model, x_abs .>= -x)
    @objective(model, Min, sum(x_abs[i] for i in 1:d));
    set_silent(model)
    optimize!(model)
    x = value.(x);
    X[:,j] = x;
    print(j)
    print(", ")
end

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 

In [10]:
model = Model(COSMO.Optimizer)

set_optimizer_attribute(model, "max_iter",500)
set_optimizer_attribute(model, "eps_prim_inf",1e-6)
set_optimizer_attribute(model, "eps_abs",1e-6)


@variable(model, L[i] <= x[i = 1:d*k] <= U[i],start=vec(X)[i])
@variable(model,  norm_x[1:d])
@variable(model, norm_c[1:k])

for i in 1:k
    @constraint(model,[norm_c;S*x[i*d-d+1:i*d]] in SecondOrderCone())
end

for i in 1:d
    @constraint(model,[norm_x[i]; x[[j*d-d+i for j in 1:k]]] in SecondOrderCone())
end

@objective(model,Min,(1/lambda)*sum(norm_x[i] for i in 1:d)+sum(norm_c[i] for i in 1:k))
optimize!(model)

------------------------------------------------------------------
          COSMO v0.8.1 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2021
------------------------------------------------------------------

Problem:  x ∈ R^{227306},
          constraints: A ∈ R^{783956x227306} (1654356 nnz),
          matrix size to factor: 1011262x1011262,
          Floating-point precision: Float64
Sets:     Nonnegatives) of dim: 445600
          SecondOrderCone) of dim: 2222
          SecondOrderCone) of dim: 2222
          SecondOrderCone) of dim: 2222
          SecondOrderCone) of dim: 2222
          ... and 4502 more
Settings: ϵ_abs = 1.0e-06, ϵ_rel = 1.0e-05,
          ϵ_prim_inf = 1.0e-06, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 500,
          scaling iter = 10 (on),
          check termination every 25 iter,
          check infeasibility every 40 iter,
          KKT system s

In [41]:
writedlm("output.txt", Matrix(reshape(value.(x),(d,k))), ',')